In [1]:
import numpy as np
import os
import pandas as pd
import SimpleFunctions as SF
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
NPoints = 21
CostData = pd.DataFrame(dtype='float64')
Values = pd.DataFrame(dtype='float64')
# ValuesRange = pd.read_csv('CostFunctionGrid/Range.txt', sep=',', decimal='.')


for Iteration in range(1,2,1):
    CostFile = pd.read_csv('CostFunctionGrid/Results' + str(Iteration) + '.csv', sep=',', decimal='.')    
    CostData = CostData.append(CostFile, ignore_index=True)

In [ ]:
CostData

In [3]:
# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]


NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

# Folder for the results
Optim_Path = os.path.join('OptimizationResults', BCsType, ConstitutiveModel)

OptiName = str(NumberElements) + 'Elements.txt'
MarcName = str(NumberElements) + 'Elements-M.txt'

# Cost Function Data
CostData = pd.read_csv('CostFunctionGrid/Results.csv', sep=',', decimal='.') 
OptiData = pd.read_csv(os.path.join(Optim_Path, OptiName), sep=' ', decimal='.')
MarcData = pd.read_csv(os.path.join(Optim_Path, MarcName), sep=' ', decimal='.')

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCostNormMax'].idxmin()])

Nus    = CostData.Nu
Mus    = CostData.Mu
Alphas = CostData.Alpha
Costs  = CostData.TotalCostNormMax

DM, MuM, AlphaM = 1.026, 0.458, -59.367
NuM = (3-DM*MuM)/(DM*MuM+6)
Marcdf = pd.DataFrame([NuM, MuM, AlphaM])

Threshold = 1E-0
Subsampling = 1

Vmin = Costs.min()
Vmax = Threshold * (Costs.max() - Costs.min())

# 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs[0::Subsampling], cmap=plt.gray(), marker = '.', vmin=Vmin, vmax=Vmax, label='Points tested')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[CostData['TotalCostNormMax'].idxmin()]], [Mus[CostData['TotalCostNormMax'].idxmin()]], [Alphas[CostData['TotalCostNormMax'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m', label='Budday starting point')
ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m', label='Optimization path')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
ax.plot([MarcData.Nu[0]], [MarcData.Mu[0]], [MarcData.Alpha[0]], marker = 'o', color='c', label='Marc starting point')
ax.plot(MarcData.Nu, MarcData.Mu, MarcData.Alpha, marker = None, color='c', label='Optimization path')
ax.plot([MarcData.Nu[len(MarcData)-1]], [MarcData.Mu[len(MarcData)-1]], [MarcData.Alpha[len(MarcData)-1]], marker = 'o', color='b', label='Optimization result')
ax.plot(Marcdf.loc[0], Marcdf.loc[1], Marcdf.loc[2], marker = 'o', color='g', label='Marc optimization result')
ax.legend()
plt.show()

Min cost point for:
Alpha                           -36
BCsType                       Fixed
CompressionCostNoNorm     0.0386284
CompressionCostNormMax    0.0691508
CompressionCostStep        0.875571
ConstitutiveModel             Ogden
Mu                              0.3
Nu                            0.315
NumberElements                   20
SimpleShearCostNoNorm      0.010957
SimpleShearCostNormMax     0.157149
SimpleShearCostStep         2.57893
TensileCostNoNorm         0.0246718
TensileCostNormMax         0.988543
TensileCostStep             237.725
TotalCostNoNorm           0.0247524
TotalCostNormMax           0.404947
TotalCostStep               80.3932
Name: 6488, dtype: object


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# Filtering

MaxRow = CostData['TotalCostNormMax'].idxmax()

Threshold = 1
MaxCost = CostData.TotalCostNormMax.max()

FilteredCost = CostData.dropna()

# for Point in range(len(FilteredCost)):
#     CostValue = FilteredCost.TotalCostNormMax[Point]
#     if CostValue > MaxCost * Threshold:
#         FilteredCost = FilteredCost.drop([Point])
        
# Filtered 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = FilteredCost.Nu
Mus    = FilteredCost.Mu
Alphas = FilteredCost.Alpha
Costs  = FilteredCost.TotalCostNormMax

# img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.', label='Points tested')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[FilteredCost['TotalCostNormMax'].idxmin()]], [Mus[FilteredCost['TotalCostNormMax'].idxmin()]], [Alphas[FilteredCost['TotalCostNormMax'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
ax.legend()
plt.show()

In [ ]:
CostData